# Exploratory logit models

In [ ]:
### Imports

In [ ]:
import os
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.font_manager
import seaborn as sn
from scipy.stats import zscore
import numpy as np
import statsmodels.api as sm


from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score


#matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
sn.set_style("white")

from scipy.spatial import cKDTree

csfont = {'fontname':'Adobe Garamond Pro','fontsize':30}
hfont = {'fontname':'Adobe Garamond Pro','fontsize':12}

### Read and prepare dataset

In [ ]:
ds_1500 = pd.read_parquet(r"../data/extraction_1500.parquet")

In [ ]:
df_testlogit = ds_1500[['elevator','trip_purpose', 'trip_distance', 'trip_mode', 'dem_gender', 'dem_income','dem_age','dem_education', 'dem_activity', 'dem_hou_size','acc_shopping_market', 'acc_shopping_alone']]
df_testlogit = df_testlogit[df_testlogit['dem_income'].notnull()]
for v in ['acc_shopping_market', 'acc_shopping_alone']:
    df_testlogit[v] = df_testlogit[v].fillna(0)
df_testlogit['acc_shopping'] = df_testlogit['acc_shopping_alone'] + df_testlogit['acc_shopping_market']
df_testlogit = df_testlogit.drop(columns = ['acc_shopping_market', 'acc_shopping_alone'])
df_testlogit = df_testlogit[df_testlogit['trip_purpose'] == 'Shopping']
df_testlogit['dem_education'] = df_testlogit['dem_education'].map({'Second - B':3, 'First':1, 'Third':4, 'Second - A':2, 'No':0})
df_testlogit['dem_activity'] = df_testlogit['dem_activity'].map({'Retired':0, 'Caretaker':1, 'Unemployed':0, 'Worker':1, 'Student':1})
df_testlogit['dem_gender'] = df_testlogit['dem_gender'].map({'Female':0,'Male':1})
df_testlogit = df_testlogit[df_testlogit['trip_mode'] != 'Other']
df_testlogit['trip_mode'] = df_testlogit['trip_mode'].map({'Walking':'active', 'Motor - Personal':'driving', 'Bus':'transit', 'Cycling':'active', 'Train':'transit','Motor - Shared':'driving'})
df_testlogit = df_testlogit[(df_testlogit['dem_age']>=18)&(df_testlogit['dem_age']<=65)]
df_testlogit = df_testlogit[df_testlogit['trip_mode'].isin(['active','driving'])]

### Further filtering model parametes

In [ ]:
# Driving age
data_filtered = df_testlogit[(df_testlogit['dem_age'] >= 18) & (df_testlogit['dem_age'] <= 65)]
# Balance the `trip_mode` column
active = data_filtered[data_filtered['trip_mode'] == 'active']
driving = data_filtered[data_filtered['trip_mode'] == 'driving']
# Resample the minority class to balance the dataset
min_class_count = min(len(active), len(driving))
active_resampled = resample(active, replace=False, n_samples=min_class_count, random_state=42)
driving_resampled = resample(driving, replace=False, n_samples=min_class_count, random_state=42)
# Combine the balanced dataset
data_filtered = pd.concat([active_resampled, driving_resampled])

### Fitting a model without distance contraints

In [ ]:
# Step 1: Define features and target variable
features = ['trip_distance', 'dem_gender', 'dem_income', 'dem_age','dem_education', 'dem_activity', 'dem_hou_size', 'acc_shopping']
target = 'trip_mode'
# Encode the target variable: 'active' -> 0, 'driving' -> 1
data_filtered['trip_mode_encoded'] = data_filtered['trip_mode'].map({'active': 0, 'driving': 1})
# Step 2: Scale numeric features
scaler = StandardScaler()
data_filtered[features] = scaler.fit_transform(data_filtered[features])
# Step 3: Split into training and testing sets
X = data_filtered[features]
y = data_filtered['trip_mode_encoded']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Display the shape of the training and testing sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Add a constant term for the intercept in the model
X_train_const = sm.add_constant(X_train)

# Fit the Binomial Logit Model using the training data and weights
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit()

# Display the summary of the model
result.summary()

# Prepare test data with a constant term
X_test_const = sm.add_constant(X_test)

# Predict probabilities for the test set
y_pred_prob = result.predict(X_test_const)

# Convert probabilities to binary predictions using 0.5 as threshold
y_pred = (y_pred_prob >= 0.5).astype(int)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Extract coefficients and their confidence intervals
coefficients = result.params
conf_intervals = result.conf_int()
features_with_const = ['Intercept'] + features  # Include the constant term

# Create a DataFrame for visualization
impact_df = pd.DataFrame({
    'Feature': features_with_const,
    'Coefficient': coefficients.values,
    'Lower CI': conf_intervals[0].values,
    'Upper CI': conf_intervals[1].values
}).set_index('Feature')

# Sort by absolute coefficient values for better visualization
impact_df['Abs Coefficient'] = np.abs(impact_df['Coefficient'])
impact_df = impact_df.sort_values(by='Abs Coefficient', ascending=False)

# Plot feature impacts with confidence intervals
plt.figure(figsize=(8.5, 6))
plt.errorbar(
    impact_df.index, impact_df['Coefficient'], 
    yerr=(impact_df['Coefficient'] - impact_df['Lower CI'], impact_df['Upper CI'] - impact_df['Coefficient']),
    fmt='o', capsize=5, label="Coefficient ± CI",color='r'
)
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
plt.xticks(rotation=45)
plt.title('Feature Impact on Mode Choice\nNo Distance Threshold',**csfont)
plt.ylabel('Coefficient',**hfont)
plt.xlabel('Feature',**hfont)
plt.tight_layout()
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig(r"../figures/17a_variable_importance_logit.jpg")
plt.show()


# Add a constant term for the intercept
X_train_const = sm.add_constant(X_train)

# Fit the Binomial Logit Model
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit()

print(result.summary())


### Fitting a model with distance contraints

In [ ]:
# Step 1: Filter dataset for valid rows
data_filtered = df_testlogit[(df_testlogit['dem_age'] >= 18) & (df_testlogit['dem_age'] <= 65)]
data_filtered = data_filtered[data_filtered['trip_distance']<1.5]
# Step 2: Balance the `trip_mode` column
active = data_filtered[data_filtered['trip_mode'] == 'active']
driving = data_filtered[data_filtered['trip_mode'] == 'driving']
# Resample the minority class to balance the dataset
min_class_count = min(len(active), len(driving))
active_resampled = resample(active, replace=False, n_samples=min_class_count, random_state=42)
driving_resampled = resample(driving, replace=False, n_samples=min_class_count, random_state=42)
# Combine the balanced dataset
data_filtered = pd.concat([active_resampled, driving_resampled])

# Step 1: Define features and target variable
features = ['trip_distance', 'dem_gender', 'dem_income', 'dem_age','dem_education', 'dem_activity', 'dem_hou_size', 'acc_shopping']
target = 'trip_mode'
# Encode the target variable: 'active' -> 0, 'driving' -> 1
data_filtered['trip_mode_encoded'] = data_filtered['trip_mode'].map({'active': 0, 'driving': 1})
# Step 2: Scale numeric features
scaler = StandardScaler()
data_filtered[features] = scaler.fit_transform(data_filtered[features])
# Step 3: Split into training and testing sets
X = data_filtered[features]
y = data_filtered['trip_mode_encoded']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Display the shape of the training and testing sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Add a constant term for the intercept in the model
X_train_const = sm.add_constant(X_train)

# Fit the Binomial Logit Model using the training data and weights
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit()

# Display the summary of the model
result.summary()

# Prepare test data with a constant term
X_test_const = sm.add_constant(X_test)

# Predict probabilities for the test set
y_pred_prob = result.predict(X_test_const)

# Convert probabilities to binary predictions using 0.5 as threshold
y_pred = (y_pred_prob >= 0.5).astype(int)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Extract coefficients and their confidence intervals
coefficients = result.params
conf_intervals = result.conf_int()
features_with_const = ['Intercept'] + features  # Include the constant term

# Create a DataFrame for visualization
impact_df = pd.DataFrame({
    'Feature': features_with_const,
    'Coefficient': coefficients.values,
    'Lower CI': conf_intervals[0].values,
    'Upper CI': conf_intervals[1].values
}).set_index('Feature')

# Sort by absolute coefficient values for better visualization
impact_df['Abs Coefficient'] = np.abs(impact_df['Coefficient'])
impact_df = impact_df.sort_values(by='Abs Coefficient', ascending=False)

# Plot feature impacts with confidence intervals
plt.figure(figsize=(8.5, 6))
plt.errorbar(
    impact_df.index, impact_df['Coefficient'], 
    yerr=(impact_df['Coefficient'] - impact_df['Lower CI'], impact_df['Upper CI'] - impact_df['Coefficient']),
    fmt='o', capsize=5, label="Coefficient ± CI",color='r'
)
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
plt.xticks(rotation=45)
plt.title('Feature Impact on Mode Choice\n1500 m Threshold',**csfont)
plt.ylabel('Coefficient',**hfont)
plt.xlabel('Feature',**hfont)
plt.tight_layout()
plt.grid()
plt.legend()
plt.tight_layout()
plt.savefig(r"../figures/17b_variable_importance_logit_proximity.jpg")
plt.show()

# Add a constant term for the intercept
X_train_const = sm.add_constant(X_train)

# Fit the Binomial Logit Model
logit_model = sm.Logit(y_train, X_train_const)
result = logit_model.fit()

print(result.summary())
